In [59]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

**Cargar el conjunto de datos**

In [2]:
df_store_sales = pd.read_csv('../data/raw/train.csv')
df_store_sales.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


**Cantidad de filas y columnas**

In [3]:
filas, columnas = df_store_sales.shape
print(f'Filas: {filas}\nColumnas: {columnas}')

Filas: 9800
Columnas: 18


**Estructura y tipo de datos**

In [4]:
df_store_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

Los tipos de datos son correctos en la mayoria de las columnas, con excepción de unos como:
* Order Date y Ship Date que son tipo ``objec`` cuando deberian ser de tipo Date.
* Postal Code es de tipo ``float64`` lo cual es parcialmente correcto, debearia de ser tipo `int64`


**Identificar valores nulos**

In [5]:
cantidad_valores_nulos = df_store_sales.isnull().sum()
print(f'Cantidad de valores nulos\n{cantidad_valores_nulos}')

Cantidad de valores nulos
Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country           0
City              0
State             0
Postal Code      11
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
dtype: int64


Se identificaron 11 valores nulos en la columna ``Postal Code``.

In [6]:
filas_valores_nulos = df_store_sales[df_store_sales.isnull().any(axis=1)]
filas_valores_nulos

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
2234,2235,CA-2018-104066,05/12/2018,10/12/2018,Standard Class,QJ-19255,Quincy Jones,Corporate,United States,Burlington,Vermont,NaN,East,TEC-AC-10001013,Technology,Accessories,Logitech ClearChat Comfort/USB Headset H390,205.03
5274,5275,CA-2016-162887,07/11/2016,09/11/2016,Second Class,SV-20785,Stewart Visinsky,Consumer,United States,Burlington,Vermont,NaN,East,FUR-CH-10000595,Furniture,Chairs,Safco Contoured Stacking Chairs,715.20
8798,8799,US-2017-150140,06/04/2017,10/04/2017,Standard Class,VM-21685,Valerie Mitchum,Home Office,United States,Burlington,Vermont,NaN,East,TEC-PH-10002555,Technology,Phones,Nortel Meridian M5316 Digital phone,1294.75
9146,9147,US-2017-165505,23/01/2017,27/01/2017,Standard Class,CB-12535,Claudia Bergmann,Corporate,United States,Burlington,Vermont,NaN,East,TEC-AC-10002926,Technology,Accessories,Logitech Wireless Marathon Mouse M705,99.98
9147,9148,US-2017-165505,23/01/2017,27/01/2017,Standard Class,CB-12535,Claudia Bergmann,Corporate,United States,Burlington,Vermont,NaN,East,OFF-AR-10003477,Office Supplies,Art,4009 Highlighters,8.04
9148,9149,US-2017-165505,23/01/2017,27/01/2017,Standard Class,CB-12535,Claudia Bergmann,Corporate,United States,Burlington,Vermont,NaN,East,OFF-ST-10001526,Office Supplies,Storage,Iceberg Mobile Mega Data/Printer Cart,1564.29
9386,9387,US-2018-127292,19/01/2018,23/01/2018,Standard Class,RM-19375,Raymond Messe,Consumer,United States,Burlington,Vermont,NaN,East,OFF-PA-10000157,Office Supplies,Paper,Xerox 191,79.92
9387,9388,US-2018-127292,19/01/2018,23/01/2018,Standard Class,RM-19375,Raymond Messe,Consumer,United States,Burlington,Vermont,NaN,East,OFF-PA-10001970,Office Supplies,Paper,Xerox 1881,12.28
9388,9389,US-2018-127292,19/01/2018,23/01/2018,Standard Class,RM-19375,Raymond Messe,Consumer,United States,Burlington,Vermont,NaN,East,OFF-AP-10000828,Office Supplies,Appliances,Avanti 4.4 Cu. Ft. Refrigerator,542.94
9389,9390,US-2018-127292,19/01/2018,23/01/2018,Standard Class,RM-19375,Raymond Messe,Consumer,United States,Burlington,Vermont,NaN,East,OFF-EN-10001509,Office Supplies,Envelopes,Poly String Tie Envelopes,2.04


Se identificaron las filas con valores nulos en la columna `Postal Code` (Se analiza para determinar si es posible rellenar con los datos que se tienen)

Customer ID     
- QJ-19255
- SV-20785
- VM-21685	
- CB-12535	
- RM-19375	

In [13]:
estado_codigo_postal = df_store_sales[df_store_sales['State'] == 'Vermont']
estado_codigo_postal

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
2234,2235,CA-2018-104066,05/12/2018,10/12/2018,Standard Class,QJ-19255,Quincy Jones,Corporate,United States,Burlington,Vermont,NaN,East,TEC-AC-10001013,Technology,Accessories,Logitech ClearChat Comfort/USB Headset H390,205.03
5274,5275,CA-2016-162887,07/11/2016,09/11/2016,Second Class,SV-20785,Stewart Visinsky,Consumer,United States,Burlington,Vermont,NaN,East,FUR-CH-10000595,Furniture,Chairs,Safco Contoured Stacking Chairs,715.20
8798,8799,US-2017-150140,06/04/2017,10/04/2017,Standard Class,VM-21685,Valerie Mitchum,Home Office,United States,Burlington,Vermont,NaN,East,TEC-PH-10002555,Technology,Phones,Nortel Meridian M5316 Digital phone,1294.75
9146,9147,US-2017-165505,23/01/2017,27/01/2017,Standard Class,CB-12535,Claudia Bergmann,Corporate,United States,Burlington,Vermont,NaN,East,TEC-AC-10002926,Technology,Accessories,Logitech Wireless Marathon Mouse M705,99.98
9147,9148,US-2017-165505,23/01/2017,27/01/2017,Standard Class,CB-12535,Claudia Bergmann,Corporate,United States,Burlington,Vermont,NaN,East,OFF-AR-10003477,Office Supplies,Art,4009 Highlighters,8.04
9148,9149,US-2017-165505,23/01/2017,27/01/2017,Standard Class,CB-12535,Claudia Bergmann,Corporate,United States,Burlington,Vermont,NaN,East,OFF-ST-10001526,Office Supplies,Storage,Iceberg Mobile Mega Data/Printer Cart,1564.29
9386,9387,US-2018-127292,19/01/2018,23/01/2018,Standard Class,RM-19375,Raymond Messe,Consumer,United States,Burlington,Vermont,NaN,East,OFF-PA-10000157,Office Supplies,Paper,Xerox 191,79.92
9387,9388,US-2018-127292,19/01/2018,23/01/2018,Standard Class,RM-19375,Raymond Messe,Consumer,United States,Burlington,Vermont,NaN,East,OFF-PA-10001970,Office Supplies,Paper,Xerox 1881,12.28
9388,9389,US-2018-127292,19/01/2018,23/01/2018,Standard Class,RM-19375,Raymond Messe,Consumer,United States,Burlington,Vermont,NaN,East,OFF-AP-10000828,Office Supplies,Appliances,Avanti 4.4 Cu. Ft. Refrigerator,542.94
9389,9390,US-2018-127292,19/01/2018,23/01/2018,Standard Class,RM-19375,Raymond Messe,Consumer,United States,Burlington,Vermont,NaN,East,OFF-EN-10001509,Office Supplies,Envelopes,Poly String Tie Envelopes,2.04


Se determino que los valores nulos de `Postal Code` corresponden una sola ciudad y estado (United States, Burlington, Vermont), por lo que una pequeña busqueda en internet se encontro el código postal que es `05403`.

**Valores duplicados**

In [16]:
valores_duplicados = df_store_sales.duplicated().sum()
print(f'Cantidad de valores duplicados:\n{valores_duplicados}')

Cantidad de valores duplicados:
0


No se identificaron valores duplicados en el cojunto de datos.

**Identificar errores ortográficos**

In [60]:
def detectar_palabras_unicas(nombre_columna):
    # Obtener la lista de nombres únicos y limpios
    nombres_unicos = df_store_sales[nombre_columna].dropna().unique()

    # Un umbral de similitud para considerar que dos nombres son "similares"
    UMBRAL_SIMILITUD = 90 # Porcentaje de similitud (0-100). Ajusta según tu necesidad.

    print("\nPosibles inconsistencias detectadas por similitud (Fuzzy Matching):\n")

    # Para evitar comparaciones duplicadas y auto-comparaciones
    nombres_revisados = set()

    for i, nombre1 in enumerate(nombres_unicos):
        if nombre1 in nombres_revisados:
            continue

        # Encuentra nombres similares a 'nombre1' en el resto de la lista
        # Usamos process.extractBests para obtener los mejores scores
        # filter_matches=True para no incluir el nombre exacto si el umbral es 100
        similares = process.extractBests(nombre1, nombres_unicos,
                                        scorer=fuzz.ratio, # Puedes probar fuzz.token_sort_ratio o fuzz.token_set_ratio
                                        score_cutoff=UMBRAL_SIMILITUD)

        # Filtrar para obtener solo coincidencias que no son idénticas a nombre1
        # y que no hemos procesado ya como el "primer" nombre de un par
        potenciales_errores = [
            (nombre_similar, score) for nombre_similar, score in similares
            if nombre_similar != nombre1 and nombre_similar not in nombres_revisados
        ]

        if potenciales_errores:
            print(f"'{nombre1}' (Frecuencia: {df_store_sales[nombre_columna].value_counts()[nombre1]}):")
            for nombre_similar, score in potenciales_errores:
                print(f"  - Posible inconsistencia: '{nombre_similar}' (Frecuencia: {df_store_sales[nombre_columna].value_counts()[nombre_similar]}, Similitud: {score}%)")
                nombres_revisados.add(nombre_similar) # Marca este nombre como revisado
            nombres_revisados.add(nombre1) # Marca también el nombre principal

    print("\n--- Fin de la revisión de inconsistencias por similitud ---")

In [ ]:
# Ship Mode
identificar_errores_ship_mode = df_store_sales['Ship Mode'].value_counts()
print(f'Identificación de errores de entrada: {identificar_errores_ship_mode}')

Identificación de errores de entrada: Ship Mode
Standard Class    5859
Second Class      1902
First Class       1501
Same Day           538
Name: count, dtype: int64


In [ ]:
# Customer Name
df_store_sales['customer_name_normalizado'] = df_store_sales['Customer Name'].str.lower().str.strip()
detectar_palabras_unicas('customer_name_normalizado')

# print(f'Identificación de errores de entrada: {df_store_sales['customer_name_normalizado'].value_counts().sort_values(ascending=True).head(30)}')


Posibles inconsistencias detectadas por similitud (Fuzzy Matching):


--- Fin de la revisión de inconsistencias por similitud ---


In [62]:
# Segment
identificar_errores_segment = df_store_sales['Segment'].value_counts()
print(f'Identificación de errores de entrada: {identificar_errores_segment}')

Identificación de errores de entrada: Segment
Consumer       5101
Corporate      2953
Home Office    1746
Name: count, dtype: int64


In [63]:
# Country
identificar_errores_country = df_store_sales['Country'].value_counts()
print(f'Identificación de errores de entrada: {identificar_errores_country}')

Identificación de errores de entrada: Country
United States    9800
Name: count, dtype: int64


In [66]:
# City
df_store_sales['city_normalizado'] = df_store_sales['City'].str.lower().str.strip()
detectar_palabras_unicas('city_normalizado')


Posibles inconsistencias detectadas por similitud (Fuzzy Matching):

'orland park' (Frecuencia: 1):
  - Posible inconsistencia: 'overland park' (Frecuencia: 6, Similitud: 92%)
'edmond' (Frecuencia: 2):
  - Posible inconsistencia: 'edmonds' (Frecuencia: 14, Similitud: 92%)
  - Posible inconsistencia: 'redmond' (Frecuencia: 11, Similitud: 92%)
'amarillo' (Frecuencia: 10):
  - Posible inconsistencia: 'camarillo' (Frecuencia: 5, Similitud: 94%)
'trenton' (Frecuencia: 13):
  - Posible inconsistencia: 'renton' (Frecuencia: 3, Similitud: 92%)

--- Fin de la revisión de inconsistencias por similitud ---


Las ciudades que aparecen com posibles inconsistencias no son inconsistencias son ciudades que si existen y que no representan errores.

In [68]:
# State
identificar_errores_state = df_store_sales['State'].value_counts()
print(f'Identificación de errores de entrada: {identificar_errores_state.sort_index(ascending=True)}')

Identificación de errores de entrada: State
Alabama                   61
Arizona                  223
Arkansas                  60
California              1946
Colorado                 179
Connecticut               82
Delaware                  93
District of Columbia      10
Florida                  373
Georgia                  177
Idaho                     21
Illinois                 483
Indiana                  135
Iowa                      26
Kansas                    24
Kentucky                 137
Louisiana                 41
Maine                      8
Maryland                 105
Massachusetts            135
Michigan                 253
Minnesota                 89
Mississippi               53
Missouri                  66
Montana                   15
Nebraska                  38
Nevada                    39
New Hampshire             27
New Jersey               122
New Mexico                37
New York                1097
North Carolina           247
North Dakota               7

In [69]:
# Region
identificar_errores_region = df_store_sales['Region'].value_counts()
print(f'Identificación de errores de entrada: {identificar_errores_region}')

Identificación de errores de entrada: Region
West       3140
East       2785
Central    2277
South      1598
Name: count, dtype: int64


In [70]:
# Category
identificar_errores_categoria = df_store_sales['Category'].value_counts()
print(f'Identificación de errores de entrada: {identificar_errores_categoria}')

Identificación de errores de entrada: Category
Office Supplies    5909
Furniture          2078
Technology         1813
Name: count, dtype: int64


In [72]:
# Sub-Category
identificar_errores_sub_Category = df_store_sales['Sub-Category'].value_counts()
print(f'Identificación de errores de entrada: {identificar_errores_sub_Category.sort_index(ascending=True)}')


Identificación de errores de entrada: Sub-Category
Accessories     756
Appliances      459
Art             785
Binders        1492
Bookcases       226
Chairs          607
Copiers          66
Envelopes       248
Fasteners       214
Furnishings     931
Labels          357
Machines        115
Paper          1338
Phones          876
Storage         832
Supplies        184
Tables          314
Name: count, dtype: int64


In [74]:
# Product Name
df_store_sales['product_name_normalizado'] = df_store_sales['Product Name'].str.lower().str.strip()
detectar_palabras_unicas('product_name_normalizado')


Posibles inconsistencias detectadas por similitud (Fuzzy Matching):

'hon deluxe fabric upholstered stacking chairs, rounded back' (Frecuencia: 12):
  - Posible inconsistencia: 'hon deluxe fabric upholstered stacking chairs, squared back' (Frecuencia: 2, Similitud: 93%)
'eldon expressions wood and plastic desk accessories, cherry wood' (Frecuencia: 5):
  - Posible inconsistencia: 'eldon expressions wood and plastic desk accessories, oak' (Frecuencia: 5, Similitud: 90%)
'newell 322' (Frecuencia: 6):
  - Posible inconsistencia: 'newell 32' (Frecuencia: 6, Similitud: 95%)
  - Posible inconsistencia: 'newell 324' (Frecuencia: 6, Similitud: 90%)
  - Posible inconsistencia: 'newell 342' (Frecuencia: 1, Similitud: 90%)
  - Posible inconsistencia: 'newell 327' (Frecuencia: 4, Similitud: 90%)
'xerox 1967' (Frecuencia: 5):
  - Posible inconsistencia: 'xerox 196' (Frecuencia: 8, Similitud: 95%)
  - Posible inconsistencia: 'xerox 197' (Frecuencia: 3, Similitud: 95%)
  - Posible inconsistencia: 'x

Los nombre de productos que aparcen como posible inconsistencias son variantes de otro (osea productos diferentes).

*No se encontraron errores de entrada en el conjunto de datos*